In [47]:
import pandas as pd

df = pd.read_csv("C:/Users/gokul/Downloads/hard_fix_september_11_2024.csv")

# Filter the dataset for years 2018 to 2022
filtered_df = df[(df['Year'] >= 2018) & (df['Year'] <= 2022)]

# Group by State and check if each state has data for all years from 2018 to 2022
years_range = set(range(2018, 2023))
states_with_complete_data = (
    filtered_df.groupby('State')['Year']
    .apply(set)  
    .apply(lambda years: years_range.issubset(years))  
    .loc[lambda x: x]  
    .index.tolist()  
)

print("States with data from 2018 to 2022:")
print(states_with_complete_data)

States with data from 2018 to 2022:
['Alabama', 'California', 'Colorado', 'Connecticut', 'Florida', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Louisiana', 'Maine', 'Maryland', 'Michigan', 'Minnesota', 'Missouri', 'New Hampshire', 'New Jersey', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Rhode Island', 'South Carolina', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'Washington, D.C.', 'Wisconsin']


In [48]:
states_with_complete_data = [
    'Alabama', 'California', 'Colorado', 'Connecticut', 'Florida', 'Hawaii', 'Idaho', 'Illinois', 
    'Indiana', 'Iowa', 'Louisiana', 'Maine', 'Maryland', 'Michigan', 'Minnesota', 'Missouri', 
    'New Hampshire', 'New Jersey', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
    'Oklahoma', 'Oregon', 'Rhode Island', 'South Carolina', 'Texas', 'Utah', 'Vermont', 
    'Virginia', 'Washington', 'Washington, D.C.', 'Wisconsin']

# Filter the dataset for the specified states and years
filtered_df2 = df[
    (df['State'].isin(states_with_complete_data)) &  
    (df['Year'] >= 2018) & (df['Year'] <= 2022)]

print(filtered_df2.head())

       State    Utility Name       Utility Type Service Type    Year  \
96   Alabama  City of Dothan  Municipal Utility     Electric  2018.0   
97   Alabama  City of Dothan  Municipal Utility     Electric  2018.0   
98   Alabama  City of Dothan  Municipal Utility     Electric  2018.0   
99   Alabama  City of Dothan  Municipal Utility     Electric  2018.0   
100  Alabama  City of Dothan  Municipal Utility     Electric  2018.0   

        Month  Total Disconnections  Total Reconnenctions  Total Connections  \
96    January                1565.0                 932.0            25721.0   
97   February                2293.0                1430.0            25535.0   
98      March                1999.0                1292.0            25736.0   
99      April                2000.0                1239.0            25582.0   
100       May                1603.0                1013.0            25702.0   

     Disconnection Rate  
96             0.060845  
97             0.089798  
98      

Calculate the mean disconnection rates for each period across each state to observe state wise trends.

Hypothesis Testing:

ANOVA is used to compare means of disconnection rates between the three periods:

𝐻0 - The mean disconnection rates are the same across all periods.

𝐻1 - At least one period has a different mean disconnection rate.

## ANOVA

In [49]:
import pandas as pd
from scipy.stats import f_oneway

# Load the filtered dataset
data = filtered_df2

# Categorize years into periods
def categorize_period(year):
    if year in [2018, 2019]:
        return 'Pre-COVID'
    elif year == 2020:
        return 'COVID'
    elif year in [2021, 2022]:
        return 'Post-COVID'

data['Period'] = data['Year'].apply(categorize_period)

# Group data by State and Period to calculate disconnection rates
state_trends = data.groupby(['State', 'Period'])['Disconnection Rate'].mean().reset_index()

print("Disconnection Rates by State and Period:")
print(state_trends)

# Perform one-way ANOVA for each state
state_results = []

for state in data['State'].unique():
    state_data = data[data['State'] == state]
    pre_covid_rates = state_data[state_data['Period'] == 'Pre-COVID']['Disconnection Rate']
    covid_rates = state_data[state_data['Period'] == 'COVID']['Disconnection Rate']
    post_covid_rates = state_data[state_data['Period'] == 'Post-COVID']['Disconnection Rate']
    
    # Perform ANOVA if there are rates for all periods
    if not (pre_covid_rates.empty or covid_rates.empty or post_covid_rates.empty):
        f_stat, p_value = f_oneway(pre_covid_rates, covid_rates, post_covid_rates)
        state_results.append({
            'State': state,
            'F-statistic': f_stat,
            'P-value': p_value,
            'Significant Difference': 'Yes' if p_value < 0.05 else 'No'
        })

# Convert results to a DataFrame
state_results_df = pd.DataFrame(state_results)

# Display the results
print("\nANOVA Results by State:")
print(state_results_df)


Disconnection Rates by State and Period:
               State      Period  Disconnection Rate
0            Alabama       COVID            0.052214
1            Alabama  Post-COVID            0.052646
2            Alabama   Pre-COVID            0.075043
3         California       COVID            0.001472
4         California  Post-COVID            0.000027
..               ...         ...                 ...
94  Washington, D.C.  Post-COVID            0.000792
95  Washington, D.C.   Pre-COVID            0.002457
96         Wisconsin       COVID            0.000006
97         Wisconsin  Post-COVID            0.001911
98         Wisconsin   Pre-COVID            0.001777

[99 rows x 3 columns]

ANOVA Results by State:
               State  F-statistic       P-value Significant Difference
0            Alabama    20.899400  2.960036e-07                    Yes
1         California          NaN           NaN                     No
2           Colorado          NaN           NaN               

C:\Users\gokul\AppData\Local\Temp\ipykernel_14032\1783167225.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Period'] = data['Year'].apply(categorize_period)


Why the Result is NaN:

The dataset contains missing values for disconnection rates in certain rows. When the statistical test encounters missing values, it cannot compute the test statistic properly. If one of the periods (Pre-COVID, COVID, Post-COVID) has no valid data points, the test cannot make a comparison.

In [50]:
from scipy.stats import f_oneway
import pandas as pd

# Initialize results list
state_results = []

for state in data['State'].unique():
    # Filter data for the current state
    state_data = data[data['State'] == state]
    
    # Extract rates for each period
    pre_covid_rates = state_data[state_data['Period'] == 'Pre-COVID']['Disconnection Rate'].dropna()
    covid_rates = state_data[state_data['Period'] == 'COVID']['Disconnection Rate'].dropna()
    post_covid_rates = state_data[state_data['Period'] == 'Post-COVID']['Disconnection Rate'].dropna()
    
    # Perform ANOVA if there are rates for all periods
    if not (pre_covid_rates.empty or covid_rates.empty or post_covid_rates.empty):
        # Perform ANOVA test
        f_stat, p_value = f_oneway(pre_covid_rates, covid_rates, post_covid_rates)
        
        # Calculate mean rates for each period
        pre_mean = pre_covid_rates.mean()
        covid_mean = covid_rates.mean()
        post_mean = post_covid_rates.mean()
        
        # Determine direction and magnitude of changes (convert to percentages)
        covid_vs_pre = (covid_mean - pre_mean) * 100
        post_vs_covid = (post_mean - covid_mean) * 100
        post_vs_pre = (post_mean - pre_mean) * 100
        
        # Append results to list
        state_results.append({
            'State': state,
            'F-statistic': f_stat,
            'P-value': p_value,
            'Significant Difference': 'Yes' if p_value < 0.05 else 'No',
            'Pre-COVID Mean Disconnection Rate': f"{abs(pre_mean * 100):.4f}%",  # Convert to percentage
            'COVID Mean Disconnection Rate': f"{abs(covid_mean * 100):.4f}%",  # Convert to percentage
            'Post-COVID Mean Disconnection Rate': f"{abs(post_mean * 100):.4f}%",  # Convert to percentage
            'Change COVID vs Pre-COVID': f"{'Increase' if covid_vs_pre > 0 else 'Decrease'} ({abs(covid_vs_pre):.4f}%)",
            'Change Post-COVID vs COVID': f"{'Increase' if post_vs_covid > 0 else 'Decrease'} ({abs(post_vs_covid):.4f}%)",
            'Change Post-COVID vs Pre-COVID': f"{'Increase' if post_vs_pre > 0 else 'Decrease'} ({abs(post_vs_pre):.4f}%)"
        })

# Convert results to a DataFrame
state_results_df = pd.DataFrame(state_results)

# Display the results
print("\nANOVA Results with Period Changes by State:")
print(state_results_df)



ANOVA Results with Period Changes by State:
               State  F-statistic       P-value Significant Difference  \
0            Alabama    20.899400  2.960036e-07                    Yes   
1         California    21.562746  1.452062e-09                    Yes   
2           Colorado    19.994920  8.094669e-09                    Yes   
3        Connecticut    20.035947  1.353850e-08                    Yes   
4            Florida    41.526804  3.871609e-17                    Yes   
5              Idaho     2.023592  1.440339e-01                     No   
6           Illinois    18.550385  1.928506e-08                    Yes   
7            Indiana    11.949935  9.907891e-06                    Yes   
8               Iowa     7.976987  4.081468e-04                    Yes   
9              Maine    27.286522  3.310772e-12                    Yes   
10          Maryland    45.592240  5.030421e-19                    Yes   
11          Michigan    53.146239  3.490726e-21                    

Given the dataset's characteristics, especially with missing data and potential non-normality in disconnection rates, considered non-parametric hypothesis testing methods as they do not rely on strict assumptions of normality or equal variances.

The p-value is much less than 0.05, so we reject H0(null hypothesis). There is a statistically significant difference in mean disconnection rates between the periods. At least one period has a different mean disconnection rate.

Alternative Method: Kruskal-Wallis H-Test

The Kruskal-Wallis H-test is a non-parametric alternative to ANOVA. It compares the medians of disconnection rates across multiple groups (Pre-COVID, COVID, Post-COVID) without assuming normality.

Why Kruskal-Wallis?

Handles missing data more robustly (after dropping missing values). Compares medians rather than means, which can be more appropriate for skewed data.

## Kruskal-Wallis H-Test

In [51]:
from scipy.stats import kruskal
import pandas as pd

# Initialize results list
state_results = []

for state in data['State'].unique():
    # Filter data for the current state
    state_data = data[data['State'] == state]
    
    # Extract rates for each period
    pre_covid_rates = state_data[state_data['Period'] == 'Pre-COVID']['Disconnection Rate'].dropna()
    covid_rates = state_data[state_data['Period'] == 'COVID']['Disconnection Rate'].dropna()
    post_covid_rates = state_data[state_data['Period'] == 'Post-COVID']['Disconnection Rate'].dropna()
    
    # Perform Kruskal-Wallis test if there are rates for all periods
    if not (pre_covid_rates.empty or covid_rates.empty or post_covid_rates.empty):
        # Perform Kruskal-Wallis test
        h_stat, p_value = kruskal(pre_covid_rates, covid_rates, post_covid_rates)
        
        # Calculate median rates for each period
        pre_median = pre_covid_rates.median()
        covid_median = covid_rates.median()
        post_median = post_covid_rates.median()
        
        # Determine direction and magnitude of changes
        covid_vs_pre = (covid_median - pre_median) * 100
        post_vs_covid = (post_median - covid_median) * 100
        post_vs_pre = (post_median - pre_median) * 100
        
        # Append results to list
        state_results.append({
            'State': state,
            'H-statistic': h_stat,
            'P-value': p_value,
            'Significant Difference': 'Yes' if p_value < 0.05 else 'No',
            'Pre-COVID Median Disconnection Rate': f"{abs(pre_median * 100):.4f}%",
            'COVID Median Disconnection Rate': f"{abs(covid_median * 100):.4f}%",
            'Post-COVID Median Disconnection Rate': f"{abs(post_median * 100):.4f}%",
            'Change COVID vs Pre-COVID': f"{'Increase' if covid_vs_pre > 0 else 'Decrease'} ({abs(covid_vs_pre):.4f}%)",
            'Change Post-COVID vs COVID': f"{'Increase' if post_vs_covid > 0 else 'Decrease'} ({abs(post_vs_covid):.4f}%)",
            'Change Post-COVID vs Pre-COVID': f"{'Increase' if post_vs_pre > 0 else 'Decrease'} ({abs(post_vs_pre):.4f}%)"
        })

# Convert results to a DataFrame
state_results_df = pd.DataFrame(state_results)

# Display the results
print("\nKruskal-Wallis Results with Period Changes by State (Using Medians):")
print(state_results_df)




Kruskal-Wallis Results with Period Changes by State (Using Medians):
               State  H-statistic       P-value Significant Difference  \
0            Alabama    31.850000  1.212999e-07                    Yes   
1         California   236.950230  3.522990e-52                    Yes   
2           Colorado    90.990975  1.744058e-20                    Yes   
3        Connecticut    39.225162  3.036437e-09                    Yes   
4            Florida    96.210849  1.282566e-21                    Yes   
5              Idaho     3.748203  1.534928e-01                     No   
6           Illinois    50.515058  1.073481e-11                    Yes   
7            Indiana    29.281798  4.380647e-07                    Yes   
8               Iowa    47.282477  5.404314e-11                    Yes   
9              Maine   104.442562  2.092116e-23                    Yes   
10          Maryland   146.703227  1.392512e-32                    Yes   
11          Michigan    74.777905  5.78341

## T-Test for comparison between Pre and Post Covid Periods

In [52]:
from scipy.stats import ttest_ind
import pandas as pd

# Initialize results list
state_results = []

for state in data['State'].unique():
    # Filter data for the current state
    state_data = data[data['State'] == state]
    
    # Extract rates for Pre-COVID and Post-COVID periods
    pre_covid_rates = state_data[state_data['Period'] == 'Pre-COVID']['Disconnection Rate'].dropna()
    post_covid_rates = state_data[state_data['Period'] == 'Post-COVID']['Disconnection Rate'].dropna()
    
    # Perform t-test if there are rates for both periods
    if not (pre_covid_rates.empty or post_covid_rates.empty):
        # Perform t-test
        t_stat, p_value = ttest_ind(pre_covid_rates, post_covid_rates, equal_var=False)  # Use Welch's t-test if variances are not equal
        
        # Calculate mean rates for each period
        pre_mean = pre_covid_rates.mean()
        post_mean = post_covid_rates.mean()
        
        # Determine direction and magnitude of changes
        post_vs_pre = (post_mean - pre_mean) * 100
        
        # Append results to list
        state_results.append({
            'State': state,
            'T-statistic': t_stat,
            'P-value': p_value,
            'Significant Difference': 'Yes' if p_value < 0.05 else 'No',
            'Pre-COVID Mean Disconnection Rate': f"{abs(pre_mean * 100):.4f}%",
            'Post-COVID Mean Disconnection Rate': f"{abs(post_mean * 100):.4f}%",
            'Change Post-COVID vs Pre-COVID': f"{'Increase' if post_vs_pre > 0 else 'Decrease'} ({abs(post_vs_pre):.4f}%)"
        })

# Convert results to a DataFrame
state_results_df = pd.DataFrame(state_results)

# Display the results
print("\nT-Test Results for Post-COVID vs Pre-COVID by State:")
print(state_results_df)



T-Test Results for Post-COVID vs Pre-COVID by State:
               State  T-statistic       P-value Significant Difference  \
0            Alabama     8.794306  2.736442e-10                    Yes   
1         California     7.133998  3.307336e-11                    Yes   
2           Colorado     4.831610  2.707788e-06                    Yes   
3        Connecticut     4.148093  2.926454e-04                    Yes   
4            Florida     5.697210  3.256112e-08                    Yes   
5             Hawaii    -1.247863  2.431035e-01                     No   
6              Idaho     1.167505  2.506881e-01                     No   
7           Illinois    -4.261961  2.644405e-05                    Yes   
8            Indiana     2.897871  4.131622e-03                    Yes   
9               Iowa     1.627542  1.047290e-01                     No   
10             Maine     0.559736  5.758477e-01                     No   
11          Maryland     6.376917  4.788028e-10           

At least one of the periods has a significantly different median disconnection rate compared to the others. This does not specify which periods differ; it only indicates that a difference exists.

We reject null hypothesis here. At least one period has a different median disconnection rate.

Next Steps:

Perform post-hoc pairwise comparisons to determine which periods (Pre-COVID vs COVID, COVID vs Post-COVID, etc.) differ from each other.

Dunn’s Test is commonly used for post-hoc pairwise comparisons after Kruskal-Wallis. It adjusts for multiple comparisons to control the family-wise error rate.

## Dunn’s Test

In [41]:
pip install scikit-posthocs

Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
import scikit_posthocs as sp
import numpy as np

# Perform Dunn's test for each state
state_dunn_results = {}

for state in data['State'].unique():
    # Filter data for the current state
    state_data = data[data['State'] == state]
    
    # Drop NaN values for each period
    pre_covid_rates = state_data[state_data['Period'] == 'Pre-COVID']['Disconnection Rate'].dropna()
    covid_rates = state_data[state_data['Period'] == 'COVID']['Disconnection Rate'].dropna()
    post_covid_rates = state_data[state_data['Period'] == 'Post-COVID']['Disconnection Rate'].dropna()
    
    # Check if all periods have data
    if len(pre_covid_rates) > 0 and len(covid_rates) > 0 and len(post_covid_rates) > 0:
        # Combine the data into one array
        all_rates = np.concatenate([pre_covid_rates, covid_rates, post_covid_rates])
        
        # Create a corresponding array of group labels
        groups = (
            ['Pre-COVID'] * len(pre_covid_rates) +
            ['COVID'] * len(covid_rates) +
            ['Post-COVID'] * len(post_covid_rates)
        )
        
        # Create a DataFrame for Dunn's test
        df = pd.DataFrame({
            'Disconnection Rate': all_rates,
            'Period': groups
        })
        
        # Run Dunn's test
        dunn_results = sp.posthoc_dunn(df, val_col='Disconnection Rate', group_col='Period', p_adjust='bonferroni')
        
        # Store the results
        state_dunn_results[state] = dunn_results

# Display results for each state
for state, results in state_dunn_results.items():
    print(f"\nDunn's Test Results for {state}:")
    print(results)


Dunn's Test Results for Alabama:
               COVID    Post-COVID     Pre-COVID
COVID       1.000000  9.042277e-01  4.829161e-04
Post-COVID  0.904228  1.000000e+00  4.105921e-07
Pre-COVID   0.000483  4.105921e-07  1.000000e+00

Dunn's Test Results for California:
                   COVID    Post-COVID     Pre-COVID
COVID       1.000000e+00  8.200807e-02  1.194004e-23
Post-COVID  8.200807e-02  1.000000e+00  2.001470e-47
Pre-COVID   1.194004e-23  2.001470e-47  1.000000e+00

Dunn's Test Results for Colorado:
                   COVID    Post-COVID     Pre-COVID
COVID       1.000000e+00  3.758817e-03  1.728064e-19
Post-COVID  3.758817e-03  1.000000e+00  2.489778e-11
Pre-COVID   1.728064e-19  2.489778e-11  1.000000e+00

Dunn's Test Results for Connecticut:
                   COVID  Post-COVID     Pre-COVID
COVID       1.000000e+00    0.003707  1.163413e-09
Post-COVID  3.707018e-03    1.000000  2.115444e-05
Pre-COVID   1.163413e-09    0.000021  1.000000e+00

Dunn's Test Results for Florida

States like California, New York, and Michigan exhibited significant differences across all pairwise comparisons, especially between Pre-COVID and Post-COVID/COVID periods, suggesting substantial shifts in disconnection rates during the pandemic. Conversely, states such as Texas, Idaho, and Utah showed no significant pairwise differences, reinforcing the conclusion that their disconnection rates remained consistent across the three periods.